In [4]:
import numpy as np
import random
from sklearn.cluster import KMeans
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
import time



In [5]:
def numpy_pos(n_agents, n_targets):
    max_size = n_targets * 2
    pos = (random.randint(0, max_size), random.randint(0, max_size))
    
    agents_pos = [(random.randint(0, max_size), random.randint(0, max_size)) for _ in range(n_agents)]
    finished_pos = [(random.randint(0, max_size), random.randint(0, max_size)) for _ in range(n_agents)]
    targets_pos = [(random.randint(0, max_size), random.randint(0, max_size)) for _ in range(n_targets)]
    
    agents_pos = np.array(agents_pos)
    finished_pos = np.array(finished_pos)
    targets_pos = np.array(targets_pos)
    
    return agents_pos, finished_pos, targets_pos

In [6]:
def list_pos(n_agents, n_targets):
    max_size = n_targets * 2
    pos = (random.randint(0, max_size), random.randint(0, max_size))
    
    agents_pos = [(random.randint(0, max_size), random.randint(0, max_size)) for _ in range(n_agents)]
    finished_pos = [(random.randint(0, max_size), random.randint(0, max_size)) for _ in range(n_agents)]
    targets_pos = [(random.randint(0, max_size), random.randint(0, max_size)) for _ in range(n_targets)]
    
    return agents_pos, finished_pos, targets_pos

In [7]:
def kmeans_clust(n_agents, targets_pos):
    kmeans = KMeans(n_clusters=n_agents, random_state=0)
    kmeans.fit(targets_pos)

    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    
    return labels, centers

In [8]:
def zip_clusters(agents, targets, labels):
    clusters = [[] for _ in range(len(agents))]
    for target, label in zip(targets, labels):
        clusters[label].append(target)
    return clusters

In [9]:
def closest_centers(agents_pos, centers, clusters):
    # 거리 행렬 계산 (agents x centers)
    cost_matrix = np.linalg.norm(agents_pos[:, np.newaxis, :] - centers[np.newaxis, :, :], axis=2)

    # 헝가리 알고리즘으로 최소 거리 매칭
    agent_indices, center_indices = linear_sum_assignment(cost_matrix)
    
    closest_clusters = [clusters[i] for i in center_indices]
    
    return agent_indices, center_indices, closest_clusters

In [10]:
def compute_euclidean_distance_matrix(locations):
    n = len(locations)
    matrix = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            if i != j:
                dx, dy = locations[i] - locations[j]
                matrix[i][j] = int(np.hypot(dx, dy))  # 소수점 줄이기 위해 스케일 업
    return matrix

In [11]:
def dist_matrix(closest_clusters, agents_pos, finished_pos):
    agents_goals = []
    for i in range(len(closest_clusters)):
        agent_goal = [agents_pos[i].tolist(), finished_pos[i].tolist()]
        for target in closest_clusters[i]:
            x, y = target
            agent_goal.append([x, y])
        agents_goals.append(agent_goal)  

    distance_matrix = []
    for locations in agents_goals:
        locations = np.array(locations)
        matrix = compute_euclidean_distance_matrix(locations)
        matrix = matrix.tolist()
        distance_matrix.append(matrix)
    
    return distance_matrix

In [12]:
def create_data_model_for_clust(distance_matrix):
    data = {}
    data["distance_matrix"] = distance_matrix
    data["num_vehicles"] = 1
    data["starts"] = [0]
    data["ends"] = [1]
    return data

def create_data_models_for_clust(distance_matrices):
    data_matrix = []
    for distance_matrix in distance_matrices:
        data = create_data_model_for_clust(distance_matrix)
        data_matrix.append(data)
    return data_matrix

In [13]:
def print_solution(data, manager, routing, solution):
    print(f"Objective: {solution.ObjectiveValue()}")
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        print(f"Route for vehicle {vehicle_id}: {route}")


In [14]:
# def print_solution(data, manager, routing, solution):
#     print(f"Objective: {solution.ObjectiveValue()}")
#     for vehicle_id in range(data["num_vehicles"]):
#         index = routing.Start(vehicle_id)
#         route = []
#         while not routing.IsEnd(index):
#             route.append(manager.IndexToNode(index))
#             index = solution.Value(routing.NextVar(index))
#         route.append(manager.IndexToNode(index))
#         print(f"Route for vehicle {vehicle_id}: {route}")

In [15]:
def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  vehicle_id = 0
  route_distance = 0
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      previous_index = index
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
      route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
    routes.append(route)
  return routes, route_distance

In [16]:
def make_solutions(closest_clusters, agents_pos, finished_pos, max_dist):
    distance_matrices = dist_matrix(closest_clusters, agents_pos, finished_pos)
    """Entry point of the program."""
    # Instantiate the data problem.
    data_matrix = create_data_models_for_clust(distance_matrices)
    
    solutions = []
    managers, routings = [], []
    routes, routes_distance = [], []

    for data in data_matrix:

        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(
            len(data["distance_matrix"]), data["num_vehicles"], data["starts"], data["ends"]
        )

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # Create and register a transit callback.
        def distance_callback(from_index, to_index):
            """Returns the distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            # print(f"[DEBUG] from_node: {from_node}, to_node: {to_node}, matrix size: {len(data['distance_matrix'])}")
            if from_node >= len(data['distance_matrix']) or to_node >= len(data['distance_matrix']):
                print(f"[DEBUG] from_node: {from_node}, to_node: {to_node}, matrix size: {len(data['distance_matrix'])}")
            return data["distance_matrix"][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Add Distance constraint.
        dimension_name = "Distance"
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            max_dist,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name,
        )
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        )

        # Solve the problem.
        solution = routing.SolveWithParameters(search_parameters)
        managers.append(manager)
        routings.append(routing)
        solutions.append(solution)
        
        if solution:
            route, route_distance = get_routes(solution, routing, manager)
            routes.append(route)
            routes_distance.append(route_distance)
        
    return data_matrix, managers, routings, solutions, routes, routes_distance

In [17]:
def clust_get_data(n_targets):
    n_targets = n_targets
    n_agents = n_targets // 30
    if n_targets % 30 > 0 :
        n_agents += 1
    max_dist = n_targets * 100

    start_time = time.time()
    agents_pos, finished_pos, targets_pos = numpy_pos(n_agents, n_targets)
    labels, centers = kmeans_clust(n_agents=n_agents, targets_pos=targets_pos)
    clusters = zip_clusters(agents=agents_pos, targets=targets_pos, labels=labels)
    agent_indices, center_indices, closest_clusters = closest_centers(agents_pos, centers, clusters)
    data_matrix, managers, routings, solutions, routes, routes_distance = make_solutions(closest_clusters, agents_pos, finished_pos, max_dist)
    end_time = time.time()
    the_time = end_time - start_time

    return routes, routes_distance, the_time

In [24]:
import sys
sys.path.append('/home/ksh-server/workspace/ICUFN/my_clustering/my_vmas/pathfinding/data')
import csv

import pandas as pd
import ast
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/ksh-server/workspace/ICUFN/my_clustering/my_vmas/pathfinding/data')
import time

# 저장된 자료 활용
df = pd.read_csv('./data/clust_data_output.csv')

# 리스트 컬럼 파싱
df['targets_per_agent'] = df['targets_per_agent'].apply(ast.literal_eval)
df['locations'] = df['locations'].apply(ast.literal_eval)

# 각각 따로 변수로 받기
n_targets = df['n_targets'].tolist()
n_agents = df['n_agents'].tolist()
times = df['time'].tolist()
targets_per_agent = df['targets_per_agent'].tolist()
locations = df['locations'].tolist()

# 저장할 파일    
file = open('./data/clust_and_tsp_time.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(['n_targets', 'n_agents', 'clust_time', 'tsp_time', 'all_time'])

for i in range(len(n_targets)):
    n_agent = n_agents[i]
    n_target = n_targets[i]
    agents_pos, targets_pos, finished_pos = np.array(locations[i][:n_agent]), np.array(locations[i][n_agent:n_agent*2]), np.array(locations[i][n_agent*2:])
    
    max_dist = n_target * 100
    start_time = time.time()
    
    # debug
    # print(n_agent, n_target)
    # print(agents_pos, targets_pos, finished_pos)
    labels, centers = kmeans_clust(n_agents=n_agent, targets_pos=targets_pos)
    clusters = zip_clusters(agents=agents_pos, targets=targets_pos, labels=labels)
    agent_indices, center_indices, closest_clusters = closest_centers(agents_pos, centers, clusters)
    clust_end_time = time.time()
    data_matrix, managers, routings, solutions, routes, routes_distance = make_solutions(closest_clusters, agents_pos, finished_pos, max_dist)
    tsp_end_time = time.time()
    clust_time = clust_end_time - start_time
    tsp_time = tsp_end_time - clust_end_time
    all_time = tsp_end_time - start_time
    
    writer.writerow([n_target, n_agent, clust_time, tsp_time, all_time])

file.close()

In [ ]:
# import sys
# sys.path.append('/home/ksh-server/workspace/ICUFN/my_clustering/my_vmas/pathfinding/data')
# import csv

# file = open('./data/only_clust_data_output.csv', 'w', newline='')
# writer = csv.writer(file)
# writer.writerow(['n_targets', 'n_agents', 'time', 'targets_per_agent'])

# total = 10000
# start = 250
# skip = 250
# repetition = 20

# for num in range(start, total+1, skip):
#     n_targets = num
#     n_agents = n_targets // 30
#     if n_targets % 30 > 0 :
#         n_agents += 1
        
#     for _ in range(repetition):
#         routes, routes_distance, the_time = clust_get_data(n_targets)
#         targets_per_agent = [len(route[0]) - 2 for route in routes]
#         writer.writerow([n_targets, n_agents, the_time, targets_per_agent])
# file.close()